## Main Goals: 
Create a cleaned development dataset that can be used to complete the modeling step of this project
- Create dummy or indicator features for categorical variables  
- Standardize the magnitude of numeric features using a scaler  
- Split into testing and training datasets

## 1. Imports packages and load the data

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer

In [2]:
jobs = pd.read_csv('data/jobs_data_step3_features.csv')

In [3]:
print(jobs.info())
print('There are {} observations and {} features in this dataset.'.format(jobs.shape[0],jobs.shape[1]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2088 entries, 0 to 2087
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Rating             1980 non-null   float64
 1   Location           2088 non-null   object 
 2   Size               2047 non-null   object 
 3   Type of ownership  2073 non-null   object 
 4   Sector             1899 non-null   object 
 5   Revenue            1474 non-null   object 
 6   Years Founded      1592 non-null   float64
 7   Min_Salary         2088 non-null   int64  
 8   Max_Salary         2088 non-null   int64  
 9   Seniority          2088 non-null   object 
 10  HQ_Same            2088 non-null   int64  
 11  SQL                2088 non-null   int64  
 12  Excel              2088 non-null   int64  
 13  Python             2088 non-null   int64  
 14  Tableau            2088 non-null   int64  
 15  R                  2088 non-null   int64  
 16  SAS                2088 

In [4]:
jobs.head()

,Rating,Location,Size,Type of ownership,Sector,Revenue,Years Founded,Min_Salary,Max_Salary,Seniority,HQ_Same,SQL,Excel,Python,Tableau,R,SAS
0,3.2,NY,201 to 500 employees,Nonprofit Organization,Non-Profit,$100 to $500 million (USD),59.0,37,66,Not specified,1,1,0,1,0,1,0
1,3.8,NY,10000+ employees,Nonprofit Organization,Health Care,$2 to $5 billion (USD),127.0,37,66,Not specified,1,1,1,0,0,1,1
2,3.4,NY,1001 to 5000 employees,Company - Private,Information Technology,NaN,17.0,37,66,Senior,1,1,1,0,1,0,0
3,4.1,NY,201 to 500 employees,Subsidiary or Business Segment,Information Technology,$50 to $100 million (USD),18.0,37,66,Not specified,0,1,0,0,1,0,0
4,3.9,NY,501 to 1000 employees,Company - Private,"Arts, Entertainment & Recreation",$100 to $500 million (USD),11.0,37,66,Not specified,1,1,1,1,0,0,0


## 2.Train/Test Split

In [5]:
# Split data to predict Min_Salary
X_train, X_test, y_train1, y_test1 = train_test_split(jobs.drop(columns=['Min_Salary','Max_Salary']), 
                                                    jobs.Min_Salary, test_size=0.3, random_state=47)
# Split data to predict Max_Salary
X_train, X_test, y_train2, y_test2 = train_test_split(jobs.drop(columns=['Min_Salary','Max_Salary']), 
                                                    jobs.Max_Salary, test_size=0.3, random_state=47)

In [6]:
X_train.shape, X_test.shape

((1461, 15), (627, 15))

## 3. Pipelines 

Create pipelines to： 
- impute missing numerical values using their median
- scale the numerical data to zero mean and unit variance
- impute missing categorical data using the most frequent values
- train a linear regression model  
Note: The EDA that we did earlier suggests a positive linear correlation between the features 'Size' and 'Years Founded'. Therefore, we will try using 'Size' to impute the missing values in the column of 'Years Founded' here.

In [7]:
# check the number of missing values for each column
jobs.isnull().sum().sort_values(ascending=False)

Revenue              614
Years Founded        496
Sector               189
Rating               108
Size                  41
Type of ownership     15
Min_Salary             0
Location               0
SAS                    0
R                      0
Seniority              0
HQ_Same                0
SQL                    0
Excel                  0
Python                 0
Tableau                0
Max_Salary             0
dtype: int64

In [8]:
# group jobs data by size and compare Years Founded
years_bySize = jobs.groupby('Size')['Years Founded'].mean()
years_bySize.sort_values(ascending=True)

Size
1 to 50 employees          15.357143
51 to 200 employees        16.078689
201 to 500 employees       24.837321
501 to 1000 employees      26.572917
1001 to 5000 employees     36.654088
5001 to 10000 employees    50.321839
10000+ employees           90.179837
Name: Years Founded, dtype: float64

It seems obvious that the larger the company, the longest history it has.

In [9]:
# check the rows with missing value in the column 'Size'
jobs.loc[jobs.Size.isnull()==True].head()

,Rating,Location,Size,Type of ownership,Sector,Revenue,Years Founded,Min_Salary,Max_Salary,Seniority,HQ_Same,SQL,Excel,Python,Tableau,R,SAS
21,NaN,NY,NaN,Company - Public,NaN,NaN,NaN,37,66,Not specified,0,1,0,1,1,1,0
82,NaN,NY,NaN,Company - Private,NaN,NaN,NaN,51,88,Not specified,0,1,0,0,1,0,0
143,3.4,NY,NaN,Company - Private,NaN,NaN,NaN,59,85,Not specified,0,0,0,0,0,0,0
170,NaN,NY,NaN,Company - Private,NaN,NaN,NaN,43,76,Not specified,1,1,1,0,1,0,0
257,3.0,NJ,NaN,Company - Public,NaN,NaN,NaN,84,90,Not specified,0,1,0,0,0,0,0


It shows that when the info about size of an observation is missing, they usually don't have infomation about sector, revenue, and years founded as well. There are 41 rows with missing Size, which is a small amount compared to 2088, the total number of rows. Threrfore, I decide to drop these 41 rows.

In [10]:
# Drop the rows with missing size info
# Impute Years_Founded using Size

jobs.dropna(subset=['Size'],axis=0,inplace=True)
jobs['Years Founded']=jobs.groupby("Size")['Years Founded'].transform(lambda x: x.fillna(x.mean()))
jobs['Years Founded'].value_counts()

15.357143     235
16.078689     115
12.000000      80
20.000000      54
24.000000      54
             ... 
202.000000      1
102.000000      1
170.000000      1
89.000000       1
112.000000      1
Name: Years Founded, Length: 166, dtype: int64

In [11]:
jobs.isnull().sum().sort_values(ascending=False)

Revenue              576
Sector               156
Rating                87
Type of ownership     10
Min_Salary             0
Location               0
Size                   0
Years Founded          0
SAS                    0
R                      0
Seniority              0
HQ_Same                0
SQL                    0
Excel                  0
Python                 0
Tableau                0
Max_Salary             0
dtype: int64

Since 'Size' and 'Years Founded' are colinear, we will drop the feature 'Size' after using it to impute the 'Years Founded' column.

In [12]:
# drop the feature 'Size'
jobs.drop('Size',axis=1,inplace=True)

#### Impute numeric missing values with median & Impute categorical data with most frequent values


In [13]:
jobs.columns

Index(['Rating', 'Location', 'Type of ownership', 'Sector', 'Revenue',
       'Years Founded', 'Min_Salary', 'Max_Salary', 'Seniority', 'HQ_Same',
       'SQL', 'Excel', 'Python', 'Tableau', 'R', 'SAS'],
      dtype='object')

In [30]:
categorical_features = ['HQ_Same', 'SQL', 'Excel','Python','Tableau','R','SAS',
                 'Location','Type of ownership','Sector','Revenue','Seniority']
numerical_features = ['Rating', 'Years Founded']
preprocess = make_column_transformer(
    ( make_pipeline(SimpleImputer(strategy='median'), StandardScaler()),numerical_features),
    ( make_pipeline(SimpleImputer(strategy="most_frequent"),OneHotEncoder()),categorical_features)
)

In [31]:
pipe = make_pipeline(
    preprocess,
    LinearRegression()
)

In [32]:
pipe.fit(X_train, y_train1)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Rating', 'Years Founded']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                               

In [17]:
# df=make_pipeline(preprocess).fit_transform(X_train)

In [33]:
y_tr_pred1 = pipe.predict(X_train)
y_te_pred1 = pipe.predict(X_test)

#### Assess performance on the train and test set

##### model performance on predicting minimum salary 

In [34]:
r2_score(y_train1, y_tr_pred1), r2_score(y_test1, y_te_pred1)

(0.33087668996705977, 0.26915481972632405)

In [35]:
mean_absolute_error(y_train1, y_tr_pred1), mean_absolute_error(y_test1, y_te_pred1)

(12.065175418083749, 11.764316238434038)

In [36]:
mean_squared_error(y_train1, y_tr_pred1), mean_squared_error(y_test1, y_te_pred1)

(260.68022424380956, 265.90631307601086)

##### model performance on predicting maximum salary 

In [22]:
pipe.fit(X_train, y_train2)
y_tr_pred2 = pipe.predict(X_train)
y_te_pred2 = pipe.predict(X_test)

In [23]:
r2_score(y_train2, y_tr_pred2), r2_score(y_test2, y_te_pred2)

(0.32095436902157204, 0.24052848110962965)

In [24]:
mean_absolute_error(y_train2, y_tr_pred2), mean_absolute_error(y_test2, y_te_pred2)

(18.636029980477005, 18.34578318828411)

In [25]:
mean_squared_error(y_train2, y_tr_pred2), mean_squared_error(y_test2, y_te_pred2)

(603.0178424650255, 615.0369200969942)

In [26]:
# look at the features of the observations that have the largest absolute error and determine what drives the large errors
largest_errors = abs(y_train1 - y_tr_pred1).sort_values(ascending=False).head(10)
jobs.loc[largest_errors.index]

,Rating,Location,Type of ownership,Sector,Revenue,Years Founded,Min_Salary,Max_Salary,Seniority,HQ_Same,SQL,Excel,Python,Tableau,R,SAS
653,3.7,CA,Subsidiary or Business Segment,Media,$10+ billion (USD),17.000000,113,132,Not specified,0,1,1,0,0,0,0
642,3.2,CA,Nonprofit Organization,Insurance,$2 to $5 billion (USD),43.000000,113,132,Not specified,1,1,1,0,0,0,1
1366,3.4,CA,Company - Public,Manufacturing,$2 to $5 billion (USD),17.000000,110,190,Not specified,1,1,0,0,1,0,0
635,5.0,CA,Company - Private,NaN,NaN,16.078689,113,132,Not specified,0,0,0,0,0,0,0
658,3.8,CA,Company - Private,Information Technology,NaN,4.000000,113,132,Senior,1,1,0,1,0,1,0
1374,5.0,CA,Company - Private,Information Technology,NaN,4.000000,110,190,Not specified,1,1,0,1,1,0,0
1376,4.4,CA,Company - Private,Information Technology,NaN,4.000000,110,190,Not specified,1,1,0,1,1,1,0
652,2.4,CA,Nonprofit Organization,NaN,NaN,24.837321,113,132,Not specified,1,1,0,0,0,0,0
643,3.3,CA,Company - Private,Business Services,$1 to $2 billion (USD),56.000000,113,132,Not specified,1,0,0,0,0,0,0
654,3.3,CA,Company - Private,Business Services,$1 to $2 billion (USD),56.000000,113,132,Not specified,1,0,0,0,0,0,0


In [27]:
jobs.describe()

,Rating,Years Founded,Min_Salary,Max_Salary,HQ_Same,SQL,Excel,Python,Tableau,R,SAS
count,1960.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000,2047.000000
mean,3.729898,36.799040,54.109917,89.902296,0.453835,0.585735,0.396190,0.271617,0.268197,0.176356,0.147533
std,0.669487,43.040180,19.506024,29.422334,0.497986,0.492715,0.489224,0.444902,0.443129,0.381216,0.354723
min,1.000000,1.000000,24.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.300000,15.357143,41.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.700000,21.000000,50.000000,87.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.100000,38.000000,63.000000,104.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,5.000000,322.000000,113.000000,190.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 4. Summary

Since our goal is to predict both the minimum salaray and the maximum salary of a job, we've got two dependent variables. We used the info in the 'Size' column to impute the missing values in the 'Years Founded' column since they are positively linearly correlated, and we dropped 'Size' after that. Then we imputed missing numerical values using their median, scaled the numerical data to zero mean and unit variance, imputed missing categorical data using the most frequent values, and trained a linear regression model. It showed that the model is doing prediction on minimum salary slightly better than doing predition on maximum salary.

In [29]:
# save data
datapath = 'data'
datapath_jobdata = os.path.join(datapath, 'jobs_data_step4_features.csv')
if not os.path.exists(datapath_jobdata):
    jobs.to_csv(datapath_jobdata, index=False)